In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import string
import pickle
import emoji
import scipy.stats as ss
import sklearn
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
import nltk
nltk.download('omw-1.4')
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

%matplotlib inline

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Kamen\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:
lemmatizer = WordNetLemmatizer()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

#Convert emojis to words 
with open('Emoji_Dict.p', 'rb') as fp:
        Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(mess):
    for emot in Emoji_Dict:
        mess = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), mess)
    return mess

def text_process(mess):
    
    #Convert emojis to words
    mess = convert_emojis_to_word(mess)
    mess = mess.lower() # lowercase text
    mess = REPLACE_BY_SPACE_RE.sub('', mess) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    mess = BAD_SYMBOLS_RE.sub('', mess)

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    stop = ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

    return stop

In [18]:
text_process('Hello, my name is Kamen. I hope you have a good day.')

'hello name kamen hope good day'

In [25]:
#A NEW DAY, NEW OPPORTUNITY
df_tr = pd.read_csv('train.txt',sep=';')
df_tr.rename({'i didnt feel humiliated':'text','sadness':'label'}, axis = 1, inplace=True)
df_v = pd.read_csv('val.txt', sep=';')
df_v.rename({'im feeling quite sad and sorry for myself but ill snap out of it soon':'text','sadness':'label'}, axis = 1, inplace=True)
df_te = pd.read_csv('test.csv', sep=';')
df_te.rename({'im feeling rather rotten so im not very ambitious right now':'text','sadness':'label'}, axis = 1, inplace=True)

In [30]:
df_te

,text,label
0,im updating my blog because i feel shitty,sadness
1,i never make her separate from me because i do...,sadness
2,i left with my bouquet of red and yellow tulip...,joy
3,i was feeling a little vain when i did this one,sadness
4,i cant walk into a shop anywhere where i do no...,fear
...,...,...
1994,i just keep feeling like someone is being unki...,anger
1995,im feeling a little cranky negative after this...,anger
1996,i feel that i am useful to my people and that ...,joy
1997,im feeling more comfortable with derby i feel ...,joy


In [31]:
print('Training data length: ', len(df_tr))
print('Validation data length: ', len(df_v))
print('Testing data length: ', len(df_te))

Training data length:  15999
Validation data length:  1999
Testing data length:  1999


In [32]:
#LSTM
from keras.utils.data_utils import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding, Bidirectional

In [35]:
dfs = [df_tr, df_v, df_te]
for df in dfs:
    print('Processing text data')
    df['to_check'] = df['text'].apply(text_process)
    print('Encoding y values')
    df['label']=df['label'].replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})
    print('\n')

Processing text data
Encoding y values


Processing text data
Encoding y values


Processing text data
Encoding y values




In [36]:
df_tr.head()

,text,label,to_check
0,i can go from feeling so hopeless to so damned...,3,go feeling hopeless damned hopeful around some...
1,im grabbing a minute to post i feel greedy wrong,1,im grabbing minute post feel greedy wrong
2,i am ever feeling nostalgic about the fireplac...,2,ever feeling nostalgic fireplace know still pr...
3,i am feeling grouchy,1,feeling grouchy
4,ive been feeling a little burdened lately wasn...,3,ive feeling little burdened lately wasnt sure


In [37]:
X_tr = df_tr['to_check'].values
y_tr = df_tr['label'].values
X_v = df_v['to_check'].values
y_v = df_v['label'].values
X_te = df_te['to_check'].values
y_te = df_te['label'].values
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state = 42)
print('Training data shape: ',X_tr.shape,y_tr.shape)
print('Validation data shape: ',X_v.shape,y_v.shape)

Training data shape:  (15999,) (15999,)
Validation data shape:  (1999,) (1999,)


In [38]:
#Prepare parameters
n_words = 20000 # cut texts after this 
maxlen = 80
batch_size = 128 

In [39]:
tokenizer = Tokenizer(num_words=n_words, lower=True)
tokenizer.fit_on_texts(X_tr)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 15064 unique tokens.


In [40]:
X_tr = tokenizer.texts_to_sequences(X_tr)
X_tr = pad_sequences(X_tr, maxlen=maxlen)
print('Shape of data tensor:', X_tr.shape)

Shape of data tensor: (15999, 80)


In [41]:
#One Hot Encoding Training data
y_tr=to_categorical(y_tr)
y_tr

array([[0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [42]:
#X_test
X_v = tokenizer.texts_to_sequences(X_v)
X_v = pad_sequences(X_v, maxlen=maxlen)

In [43]:
#One Hot Encode validation data
y_v = to_categorical(y_v)

In [44]:
#LSTM Model creation
model=Sequential()
model.add(Embedding(15185,64,input_length=80))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(80,return_sequences=True)))
model.add(Bidirectional(LSTM(160)))
model.add(Dense(6,activation='softmax'))
print(model.summary())
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 64)            971840    
                                                                 
 dropout (Dropout)           (None, 80, 64)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 80, 160)          92800     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 320)              410880    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 6)                 1926      
                                                                 
Total params: 1,477,446
Trainable params: 1,477,446
Non-

In [45]:
hist=model.fit(X_tr,y_tr,epochs=12,validation_data=(X_v,y_v))

Epoch 1/12
500/500 [==============================] - 608s 1s/step - loss: 1.1409 - accuracy: 0.5607 - val_loss: 0.6171 - val_accuracy: 0.7999
Epoch 2/12
500/500 [==============================] - 646s 1s/step - loss: 0.4354 - accuracy: 0.8480 - val_loss: 0.2868 - val_accuracy: 0.9010
Epoch 3/12
500/500 [==============================] - 634s 1s/step - loss: 0.2485 - accuracy: 0.9146 - val_loss: 0.2227 - val_accuracy: 0.9160
Epoch 4/12
348/500 [===================>..........] - ETA: 4:24 - loss: 0.1810 - accuracy: 0.9370

In [ ]:
"""
IMPORTANT:
Training data has to come from train.txt
Validation data has to come from val.txt
Test data has to from test.txt
"""
import matplotlib.pyplot as plt 
%matplotlib inline
plt.plot(hist.history['accuracy'],c='b',label='train')
plt.plot(hist.history['val_accuracy'],c='r',label='validation')
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(hist.history['loss'],c='orange',label='train')
plt.plot(hist.history['val_loss'],c='g',label='validation')
plt.legend(loc='upper right')
plt.show()

In [19]:
Y = pd.get_dummies(df['label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (17998, 6)


In [38]:
Y_enc[:6]

array([[0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [21]:
model = Sequential()
model.add(Embedding(n_words, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
history=model.fit(X_train, Y_train,batch_size=batch_size,epochs=12,validation_data=[X_test, Y_test])
print(history.history['loss'])
print(history.history['accuracy']) 

Epoch 1/12


ValueError: in user code:

    File "C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Kamen\AppData